<a href="https://colab.research.google.com/github/StephenOC12/3_Second_Violation_Project/blob/main/Another_copy_of_basketball_ai_automatic_detection_of_3_second_violations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# Basketball AI: Automatic Detection of 3-Second Violation

---

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/basketball-ai-automatic-detection-of-3-second-violations.ipynb)
[![roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/detect-3-second-violation-ai-basketball)

![3-second-violations-1](https://media.roboflow.com/notebooks/basketball-ai-automatic-detection-of-3-second-violations-1.png)

![3-second-violations-2](https://media.roboflow.com/notebooks/basketball-ai-automatic-detection-of-3-second-violations-2.png)


## Before you start

### Configure your API key

- Open your [`HuggingFace Settings`](https://huggingface.co/settings) page. Click `Access Tokens` then `New Token` to generate new token.
- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑).
    - Store HuggingFace Access Token under the name `HF_TOKEN`.
    - Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY")

### Select the runtime

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

### Install dependencies

**Note:** The following commands will install all the required libraries for this notebook. The `gdown` library is used to download example data from Google Drive. We use `inference-gpu` to run RF-DETR for player detection and a keypoint detection model for basketball court key points on GPUs. `transformers` is needed to run ViTPose++ for player pose estimation. We use `supervision` to parse, process, and visualize model results, and the `sports` package provides basketball-specific utilities. This setup ensures you have everything needed for object detection, keypoint detection, pose estimation, and basketball analytics on GPU.


In [ ]:
!pip install -q pandas==2.2.2

In [ ]:
!pip install -q gdown transformers
!pip install -q gdown transformers inference-gpu supervision
!pip install -q git+https://github.com/roboflow/sports.git@feat/basketball_radar

**Note:** Set the ONNX Runtime execution provider to CUDA to ensure model inference runs on the GPU.

In [ ]:
import os
os.environ["ONNXRUNTIME_EXECUTION_PROVIDERS"] = "[CUDAExecutionProvider]"

**Note:** Clone and install the SAM2 fork, which will be used for player segmentation and tracking. Installation may take a few minutes to complete, so please be patient.


In [ ]:
!git clone https://github.com/Gy920/segment-anything-2-real-time.git $HOME/segment-anything-2-real-time
%cd $HOME/segment-anything-2-real-time
!pip install -e . -q
!python setup.py build_ext --inplace

**NOTE:** SAM2 is available in 4 different model sizes ranging from the lightweight "sam2_hiera_tiny" (38.9M parameters) to the more powerful "sam2_hiera_large" (224.4M parameters).

In [ ]:
!cd $HOME/segment-anything-2-real-time/checkpoints && bash download_ckpts.sh

### Download the sample data

**Note:** Download the sample video file using `gdown`. This file will be used for testing player detection and tracking.

In [ ]:
!cd $HOME && gdown https://drive.google.com/uc?id=1EsxiyaYGj3FeXSXoK51pre5OjVGWTCSE
!cd $HOME && gdown https://drive.google.com/uc?id=1It3wz3eoGcjo6tI9OGUt3X69ZK6x5SNK
!cd $HOME && gdown https://drive.google.com/uc?id=1QRotwL1r00DT2B7b41XIJxisFsqHZN8T
!cd $HOME && gdown https://drive.google.com/uc?id=1Ka1wRUE8JwUlwDQ3aUZP9oVQ6scaYjtz

In [ ]:
from pathlib import Path

SOURCE_VIDEO_PATH = Path(HOME) / "boston-celtics-new-york-knicks-game-1-q1-04.28-04.20.mp4"
SOURCE_VIDEO_PATH

### Define some visualization utils

In [ ]:
import cv2
import numpy as np
from typing import List, Tuple

def convert_to_grayscale(image: np.ndarray) -> np.ndarray:
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(grayscale_image, cv2.COLOR_GRAY2BGR)


def replace_masked_regions(
    source_frame: np.ndarray,
    target_frame: np.ndarray,
    masks: List[np.ndarray]
) -> np.ndarray:
    result_frame = target_frame.copy()

    for mask in masks:
        if mask.ndim == 3:
            mask = np.squeeze(mask)

        if mask.dtype != bool:
            mask = mask.astype(bool)

        mask_rgb = np.repeat(mask[:, :, None], 3, axis=2)
        result_frame[mask_rgb] = source_frame[mask_rgb]

    return result_frame

## ball, number, player, referee and basket detection

In [ ]:
from inference import get_model
from google.colab import userdata
import supervision as sv

PLAYER_DETECTION_MODEL_ID = "basketball-player-detection-3-ycjdo/4"
PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key =  userdata.get("ROBOFLOW_API_KEY"))

COLOR = sv.ColorPalette.from_hex([
    "#ffff00", "#ff9b00", "#ff66ff", "#3399ff", "#ff66b2", "#ff8080",
    "#b266ff", "#9999ff", "#66ffff", "#33ff99", "#66ff66", "#99ff00"
])

In [ ]:
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

sv.plot_image(frame)

In [ ]:
box_annotator = sv.BoxAnnotator(color=COLOR, thickness=2)
label_annotator = sv.LabelAnnotator(color=COLOR, text_color=sv.Color.BLACK)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.35)[0]
detections = sv.Detections.from_inference(result)

labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence
    in zip(detections['class_name'], detections.confidence)
]

annotated_frame = frame.copy()
annotated_frame = box_annotator.annotate(
    scene=annotated_frame,
    detections=detections)
annotated_frame = label_annotator.annotate(
    scene=annotated_frame,
    detections=detections,
    labels=labels)

sv.plot_image(annotated_frame)


## work out who is ball handler

In [ ]:
# NOT NEEDED DUE TO NEW PLAYER_IN_POSSESSION CLASS, BUT GOOD TO TALK ABOUT





import numpy as np
import supervision as sv

# Run inference
result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.35)[0]
detections = sv.Detections.from_inference(result)

# Extract class ids
BALL_ID = 0
PLAYER_ID = 2

ball_detections = detections[detections.class_id == BALL_ID]
player_detections = detections[detections.class_id == PLAYER_ID]

player_in_possession_detection = None

# Get the center of all players, Calculate distance from the ball to every player
# Find the player with the minimum distance, If ball is within 50-80 pixels of player center, they have it
if len(ball_detections) > 0 and len(player_detections) > 0:
    ball_xy = ball_detections.get_anchors_coordinates(sv.Position.CENTER)[0]

    player_xy = player_detections.get_anchors_coordinates(sv.Position.CENTER)

    distances = np.linalg.norm(player_xy - ball_xy, axis=1)

    min_dist_idx = np.argmin(distances)
    min_dist = distances[min_dist_idx]

    if min_dist < 80:
        player_in_possession_detection = player_detections[[min_dist_idx]]

box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)

annotated_frame = frame.copy()

annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)

# Highlighting the player in possession (if any) and Draw a box around the ball handler
if player_in_possession_detection:
    possession_annotator = sv.BoxAnnotator(color=sv.Color.from_hex("#00FF00"), thickness=4)
    annotated_frame = possession_annotator.annotate(
        scene=annotated_frame,
        detections=player_in_possession_detection
    )

    # Add a ball handler label
    sv.draw_text(
        scene=annotated_frame,
        text="BALL HANDLER",
        text_anchor=sv.Point(
            x=player_in_possession_detection.xyxy[0][0],
            y=player_in_possession_detection.xyxy[0][1] - 20
        ),
        background_color=sv.Color.from_hex("#00FF00"),
        text_color=sv.Color.BLACK
    )

sv.plot_image(annotated_frame)

In [ ]:
# from tqdm import tqdm
# from pathlib import Path

# TARGET_VIDEO_PATH = SOURCE_VIDEO_PATH.parent / f"{SOURCE_VIDEO_PATH.stem}-detections{SOURCE_VIDEO_PATH.suffix}"
# TARGET_VIDEO_COMPRESSED_PATH = TARGET_VIDEO_PATH.parent / f"{TARGET_VIDEO_PATH.stem}-compressed{TARGET_VIDEO_PATH.suffix}"

# box_annotator = sv.BoxAnnotator(color=COLOR, thickness=2)
# label_annotator = sv.LabelAnnotator(color=COLOR, text_color=sv.Color.BLACK)

# frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
#     for index, frame in tqdm(enumerate(frame_generator)):

#         result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.5)[0]
#         detections = sv.Detections.from_inference(result)

#         labels = [
#             f"{class_name} {confidence:.2f}"
#             for class_name, confidence
#             in zip(detections['class_name'], detections.confidence)
#         ]

#         annotated_frame = frame.copy()
#         annotated_frame = convert_to_grayscale(annotated_frame)
#         annotated_frame = box_annotator.annotate(
#             scene=annotated_frame,
#             detections=detections)
#         annotated_frame = label_annotator.annotate(
#             scene=annotated_frame,
#             detections=detections,
#             labels=labels)
#         sink.write_frame(annotated_frame)

In [ ]:
# %%capture
# !ffmpeg -y -i {TARGET_VIDEO_PATH} -vcodec libx264 -crf 28 {TARGET_VIDEO_COMPRESSED_PATH}

## detect players

In [ ]:
frame = next(frame_generator)

box_annotator = sv.BoxAnnotator(color=COLOR, thickness=2)
label_annotator = sv.LabelAnnotator(color=COLOR, text_color=sv.Color.BLACK)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.35)[0]
detections = sv.Detections.from_inference(result)
detections = detections[detections.class_id == 4]

labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence
    in zip(detections['class_name'], detections.confidence)
]

annotated_frame = frame.copy()
annotated_frame = box_annotator.annotate(
    scene=annotated_frame,
    detections=detections)
annotated_frame = label_annotator.annotate(
    scene=annotated_frame,
    detections=detections,
    labels=labels)

sv.plot_image(annotated_frame)

## determine player teams

In [ ]:
from tqdm import tqdm


In [ ]:
PLAYER_DETECTION_MODEL_CONFIDENCE = 0.4
PLAYER_DETECTION_MODEL_IOU_THRESHOLD = 0.9

PLAYER_CLASS_IDS = [3,4]

STRIDE = 30
crops = []

frame_generator = sv.get_video_frames_generator(
    source_path=SOURCE_VIDEO_PATH,
    stride=STRIDE
)

for frame in tqdm(frame_generator):
    result = PLAYER_DETECTION_MODEL.infer(
        frame,
        confidence=PLAYER_DETECTION_MODEL_CONFIDENCE,
        iou_threshold=PLAYER_DETECTION_MODEL_IOU_THRESHOLD,
        class_agnostic_nms=True
    )[0]

    detections = sv.Detections.from_inference(result)
    detections = detections[np.isin(detections.class_id, PLAYER_CLASS_IDS)]

    boxes = sv.scale_boxes(xyxy=detections.xyxy, factor=0.4)

    for box in boxes:
        crops.append(sv.crop_image(frame, box))


In [ ]:
sv.plot_images_grid(
    images=crops[:100],
    grid_size=(10, 10),
    size=(10, 10)
)

In [ ]:
!pip install -q git+https://github.com/roboflow/sports.git@feat/basketball


In [ ]:
from sports.common.team import TeamClassifier

team_classifier = TeamClassifier(device="cuda")
team_classifier.fit(crops)

In [ ]:
teams = team_classifier.predict(crops)

team_0 = [crop for crop, team in zip(crops, teams) if team == 0]
team_1 = [crop for crop, team in zip(crops, teams) if team == 1]

sv.plot_images_grid(
    images=team_0[:50],
    grid_size=(5, 10),
    size=(10, 5)
)

sv.plot_images_grid(
    images=team_1[:50],
    grid_size=(5, 10),
    size=(10, 5)
)

In [ ]:
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=PLAYER_DETECTION_MODEL_CONFIDENCE, iou_threshold=PLAYER_DETECTION_MODEL_IOU_THRESHOLD, class_agnostic_nms=True)[0]
detections = sv.Detections.from_inference(result)
detections = detections[np.isin(detections.class_id, PLAYER_CLASS_IDS)]

boxes = sv.scale_boxes(xyxy=detections.xyxy, factor=0.4)
crops = [sv.crop_image(frame, box) for box in boxes]
teams = np.array(team_classifier.predict(crops))

team_0 = [crop for crop, team in zip(crops, teams) if team == 0]
team_1 = [crop for crop, team in zip(crops, teams) if team == 1]

sv.plot_images_grid(
    images=team_0[:10],
    grid_size=(1, 10),
    size=(10, 1)
)

sv.plot_images_grid(
    images=team_1[:10],
    grid_size=(1, 10),
    size=(10, 1)
)

In [ ]:
%cd $HOME/segment-anything-2-real-time

from sam2.build_sam import build_sam2_camera_predictor

SAM2_CHECKPOINT = "checkpoints/sam2.1_hiera_large.pt"
SAM2_CONFIG = "configs/sam2.1/sam2.1_hiera_l.yaml"

predictor = build_sam2_camera_predictor(SAM2_CONFIG, SAM2_CHECKPOINT)

In [ ]:
import torch
from pathlib import Path

# Ensure SOURCE_VIDEO_PATH is a Path object
SOURCE_VIDEO_PATH = Path(SOURCE_VIDEO_PATH)
TARGET_VIDEO_PATH = SOURCE_VIDEO_PATH.parent / f"{SOURCE_VIDEO_PATH.stem}-time{SOURCE_VIDEO_PATH.suffix}"

# THE FIX: Create the directory if it doesn't exist
TARGET_VIDEO_PATH.parent.mkdir(parents=True, exist_ok=True)

TARGET_VIDEO_COMPRESSED_PATH = (
    TARGET_VIDEO_PATH.parent
    / f"{TARGET_VIDEO_PATH.stem}-compressed.mp4"
)

print(f"Target path ready: {TARGET_VIDEO_PATH}")

In [ ]:
!ls {TARGET_VIDEO_PATH}

In [ ]:
!ls {TARGET_VIDEO_PATH}
TEAM_NAMES = {
    0: "Boston Celtics",
    1: "New York Knicks",
}

TEAM_COLORS = {
    "New York Knicks": "#007A33",
    "Boston Celtics": "#006BB6"
}

from __future__ import annotations

class SAM2Tracker:
    def __init__(self, predictor) -> None:
        self.predictor = predictor
        self._prompted = False

    def prompt_first_frame(self, frame: np.ndarray, detections: sv.Detections) -> None:
        if len(detections) == 0:
            raise ValueError("detections must contain at least one box")

        if detections.tracker_id is None:
            detections.tracker_id = list(range(1, len(detections) + 1))

        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            self.predictor.load_first_frame(frame)
            for xyxy, obj_id in zip(detections.xyxy, detections.tracker_id):
                bbox = np.asarray([xyxy], dtype=np.float32)
                self.predictor.add_new_prompt(
                    frame_idx=0,
                    obj_id=int(obj_id),
                    bbox=bbox,
                )

        self._prompted = True

    def propagate(self, frame: np.ndarray) -> sv.Detections:
        if not self._prompted:
            raise RuntimeError("Call prompt_first_frame before propagate")

        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            tracker_ids, mask_logits = self.predictor.track(frame)

        tracker_ids = np.asarray(tracker_ids, dtype=np.int32)
        masks = (mask_logits > 0.0).cpu().numpy()
        masks = np.squeeze(masks).astype(bool)

        if masks.ndim == 2:
            masks = masks[None, ...]

        masks = np.array([
            sv.filter_segments_by_distance(mask, relative_distance=0.03, mode="edge")
            for mask in masks
        ])

        xyxy = sv.mask_to_xyxy(masks=masks)
        detections = sv.Detections(xyxy=xyxy, mask=masks, tracker_id=tracker_ids)
        return detections

    def reset(self) -> None:
        self._prompted = False

# --------------------------WASTEFUL STUFF ABOVE, JUST TO TEST, GET RID !!!!! -------------------------------------


frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

# define team annotators

team_colors = sv.ColorPalette.from_hex([
    TEAM_COLORS[TEAM_NAMES[0]],
    TEAM_COLORS[TEAM_NAMES[1]]
])

team_mask_annotator = sv.MaskAnnotator(
    color=team_colors,
    opacity=0.5,
    color_lookup=sv.ColorLookup.INDEX
)

team_box_annotator = sv.BoxAnnotator(
    color=team_colors,
    thickness=2,
    color_lookup=sv.ColorLookup.INDEX
)

# we use RF-DETR model to aquire future SAM2 prompt

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=PLAYER_DETECTION_MODEL_CONFIDENCE, iou_threshold=PLAYER_DETECTION_MODEL_IOU_THRESHOLD)[0]
detections = sv.Detections.from_inference(result)
detections = detections[np.isin(detections.class_id, PLAYER_CLASS_IDS)]
detections.tracker_id = np.arange(1, len(detections.class_id) + 1)

# we determine the team for each player and assign a team ID to every detection

boxes = sv.scale_boxes(xyxy=detections.xyxy, factor=0.4)
crops = [sv.crop_image(frame, box) for box in boxes]
TEAMS = np.array(team_classifier.predict(crops))

# we prompt SAM2 using RF-DETR model detections

tracker = SAM2Tracker(predictor)
tracker.prompt_first_frame(frame, detections)

# we propagate tacks across all video frames

def callback(frame: np.ndarray, index: int) -> np.ndarray:
    global ATTACKING_TEAM, CANDIDATE_TEAM, CANDIDATE_COUNT

    detections = tracker.propagate(frame)

    annotated_frame = frame.copy()

    # -----------------------------
    # Detect player in possession
    # -----------------------------
    possession_detections = detections[detections.class_id == 4]

    if len(possession_detections) > 0:
        poss_tracker_id = int(possession_detections.tracker_id[0])

        if poss_tracker_id in tracker_id_to_team:
            poss_team = tracker_id_to_team[poss_tracker_id]

            if ATTACKING_TEAM is None:
                ATTACKING_TEAM = poss_team

            elif poss_team != ATTACKING_TEAM:
                # start counting consecutive frames
                if CANDIDATE_TEAM == poss_team:
                    CANDIDATE_COUNT += 1
                else:
                    CANDIDATE_TEAM = poss_team
                    CANDIDATE_COUNT = 1

                if CANDIDATE_COUNT >= SWITCH_THRESHOLD:
                    ATTACKING_TEAM = poss_team
                    CANDIDATE_TEAM = None
                    CANDIDATE_COUNT = 0
            else:
                # same team still attacking → reset candidate
                CANDIDATE_TEAM = None
                CANDIDATE_COUNT = 0

    # -----------------------------
    # Build color lookup
    # -----------------------------
    team_ids = np.array([
        tracker_id_to_team.get(tid, 0)
        for tid in detections.tracker_id
    ])

    annotated_frame = team_mask_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        custom_color_lookup=team_ids
    )

    annotated_frame = team_box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        custom_color_lookup=team_ids
    )

    # -----------------------------
    # Optional: draw attacking team label
    # -----------------------------
    if ATTACKING_TEAM is not None:
        team_name = TEAM_NAMES[ATTACKING_TEAM]
        cv2.putText(
            annotated_frame,
            f"ATTACKING: {team_name}",
            (50, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.2,
            (0, 255, 255),
            3,
            cv2.LINE_AA
        )

    return annotated_frame

sv.process_video(
    source_path=SOURCE_VIDEO_PATH,
    target_path=TARGET_VIDEO_PATH,
    callback=callback,
    show_progress=True
)

!ffmpeg -y -loglevel error \
    -i "{TARGET_VIDEO_PATH}" \
    -vcodec libx264 -crf 28 \
    "{TARGET_VIDEO_COMPRESSED_PATH}"

In [ ]:
from IPython.display import Video

In [ ]:
Video(TARGET_VIDEO_COMPRESSED_PATH, embed=True, width=720)

## detect basketball court keypoints

In [ ]:
from inference import get_model
from google.colab import userdata

COURT_DETECTION_MODEL_ID = "basketball-court-detection-2/10"
COURT_DETECTION_MODEL = get_model(model_id=COURT_DETECTION_MODEL_ID, api_key =  userdata.get("ROBOFLOW_API_KEY"))

MAGENTA_COLOR = sv.Color.from_hex('#FF1493')
CYAN_COLOR = sv.Color.from_hex('#00BFFF')

In [ ]:
vertex_annotator = sv.VertexAnnotator(color=MAGENTA_COLOR, radius=8)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = COURT_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
key_points = sv.KeyPoints.from_inference(result)

annotated_frame = frame.copy()
annotated_frame = vertex_annotator.annotate(
    scene=annotated_frame,
    key_points=key_points)

sv.plot_image(annotated_frame)

In [ ]:
vertex_annotator = sv.VertexAnnotator(color=MAGENTA_COLOR, radius=8)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = COURT_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
key_points = sv.KeyPoints.from_inference(result)

keypoint_mask = key_points.confidence[0] > 0.5
key_points.xy = key_points.xy[:, keypoint_mask, :]
key_points.confidence = key_points.confidence[:, keypoint_mask]

annotated_frame = frame.copy()
annotated_frame = vertex_annotator.annotate(
    scene=annotated_frame,
    key_points=key_points)

sv.plot_image(annotated_frame)

## project lines and missing points

In [ ]:
from sports.common.view import ViewTransformer
from sports.configs.basketball import BasketballCourtConfiguration

KEYPOINT_CONFIDENCE_THRESHOLD = 0.5
DETECTION_CONFIDENCE = 0.3

CONFIG = BasketballCourtConfiguration()
EDGES = [(p1 + 1, p2 + 1) for p1, p2 in CONFIG.edges]

detected_vertex_annotator = sv.VertexAnnotator(color=MAGENTA_COLOR, radius=8)
projected_vertex_annotator = sv.VertexAnnotator(color=CYAN_COLOR, radius=8)

projected_edge_annotator = sv.EdgeAnnotator(
    color=CYAN_COLOR,
    thickness=2,
    edges=EDGES
)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = COURT_DETECTION_MODEL.infer(frame, confidence=DETECTION_CONFIDENCE)[0]
key_points = sv.KeyPoints.from_inference(result)
filter = key_points.confidence[0] > KEYPOINT_CONFIDENCE_THRESHOLD

filtered_xy = key_points.xy[:, filter, :]
filtered_confidence = key_points.confidence[:, filter]

detected_key_points = sv.KeyPoints(
    xy=filtered_xy,
    confidence=filtered_confidence,
)

source_reference_xy = np.array(CONFIG.vertices)[filter]
target_reference_xy = detected_key_points.xy[0]

transformer = ViewTransformer(
    source=source_reference_xy,
    target=target_reference_xy
)

source_xy = np.array(CONFIG.vertices)
target_xy = transformer.transform_points(points=source_xy)

projected_key_points = sv.KeyPoints(xy=target_xy[np.newaxis, ...])

annotated_frame = frame.copy()
annotated_frame = projected_edge_annotator.annotate(
    scene=annotated_frame,
    key_points=projected_key_points)
annotated_frame = projected_vertex_annotator.annotate(
    scene=annotated_frame,
    key_points=projected_key_points)
annotated_frame = detected_vertex_annotator.annotate(
    scene=annotated_frame,
    key_points=detected_key_points)

sv.plot_image(annotated_frame)

## detect the paint

In [ ]:
from sports.common.view import ViewTransformer
from sports.configs.basketball import BasketballCourtConfiguration

LEFT_PAINT_INDICES = [2, 3, 11, 9]
RIGHT_PAINT_INDICES = [21, 23, 30, 29]
POLYGONS_INDICES = [LEFT_PAINT_INDICES, RIGHT_PAINT_INDICES]
KEYPOINT_CONFIDENCE_THRESHOLD = 0.5
DETECTION_CONFIDENCE = 0.3

CONFIG = BasketballCourtConfiguration()
EDGES = [(p1 + 1, p2 + 1) for p1, p2 in CONFIG.edges]

detected_vertex_annotator = sv.VertexAnnotator(color=MAGENTA_COLOR, radius=8)
projected_vertex_annotator = sv.VertexAnnotator(color=CYAN_COLOR, radius=8)

projected_edge_annotator = sv.EdgeAnnotator(
    color=CYAN_COLOR,
    thickness=2,
    edges=EDGES
)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = COURT_DETECTION_MODEL.infer(frame, confidence=DETECTION_CONFIDENCE)[0]
key_points = sv.KeyPoints.from_inference(result)
filter = key_points.confidence[0] > KEYPOINT_CONFIDENCE_THRESHOLD

filtered_xy = key_points.xy[:, filter, :]
filtered_confidence = key_points.confidence[:, filter]

detected_key_points = sv.KeyPoints(
    xy=filtered_xy,
    confidence=filtered_confidence,
)

source_reference_xy = np.array(CONFIG.vertices)[filter]
target_reference_xy = detected_key_points.xy[0]

transformer = ViewTransformer(
    source=source_reference_xy,
    target=target_reference_xy
)

source_xy = np.array(CONFIG.vertices)
target_xy = transformer.transform_points(points=source_xy)

projected_key_points = sv.KeyPoints(xy=target_xy[np.newaxis, ...])

annotated_frame = frame.copy()

for indices in POLYGONS_INDICES:
    polygon = target_xy[indices].astype(int)

    annotated_frame = sv.draw_filled_polygon(
        scene=annotated_frame,
        polygon=polygon,
        color=CYAN_COLOR,
        opacity=0.6
    )

annotated_frame = projected_edge_annotator.annotate(
    scene=annotated_frame,
    key_points=projected_key_points)
annotated_frame = projected_vertex_annotator.annotate(
    scene=annotated_frame,
    key_points=projected_key_points)
annotated_frame = detected_vertex_annotator.annotate(
    scene=annotated_frame,
    key_points=detected_key_points)

sv.plot_image(annotated_frame)

In [ ]:
# from tqdm import tqdm
# from pathlib import Path

# from sports.common.view import ViewTransformer
# from sports.configs.basketball import BasketballCourtConfiguration

# TARGET_VIDEO_PATH = SOURCE_VIDEO_PATH.parent / f"{SOURCE_VIDEO_PATH.stem}-paint{SOURCE_VIDEO_PATH.suffix}"
# TARGET_VIDEO_COMPRESSED_PATH = TARGET_VIDEO_PATH.parent / f"{TARGET_VIDEO_PATH.stem}-compressed{TARGET_VIDEO_PATH.suffix}"

# LEFT_PAINT_INDICES = [2, 3, 11, 9]
# RIGHT_PAINT_INDICES = [21, 23, 30, 29]
# POLYGONS_INDICES = [LEFT_PAINT_INDICES, RIGHT_PAINT_INDICES]
# KEYPOINT_CONFIDENCE_THRESHOLD = 0.5
# DETECTION_CONFIDENCE = 0.3

# CONFIG = BasketballCourtConfiguration()
# EDGES = [(p1 + 1, p2 + 1) for p1, p2 in CONFIG.edges]

# detected_vertex_annotator = sv.VertexAnnotator(color=MAGENTA_COLOR, radius=8)
# projected_vertex_annotator = sv.VertexAnnotator(color=CYAN_COLOR, radius=8)

# projected_edge_annotator = sv.EdgeAnnotator(
#     color=CYAN_COLOR,
#     thickness=2,
#     edges=EDGES
# )

# frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
#     for index, frame in tqdm(enumerate(frame_generator)):

#         result = COURT_DETECTION_MODEL.infer(frame, confidence=DETECTION_CONFIDENCE)[0]
#         key_points = sv.KeyPoints.from_inference(result)
#         filter = key_points.confidence[0] > KEYPOINT_CONFIDENCE_THRESHOLD
#         detected_key_points = key_points[:, filter]

#         source_reference_xy = np.array(CONFIG.vertices)[filter]
#         target_reference_xy = detected_key_points.xy[0]

#         transformer = ViewTransformer(
#             source=source_reference_xy,
#             target=target_reference_xy
#         )

#         source_xy = np.array(CONFIG.vertices)
#         target_xy = transformer.transform_points(points=source_xy)

#         projected_key_points = sv.KeyPoints(xy=target_xy[np.newaxis, ...])

#         annotated_frame = frame.copy()
#         annotated_frame = convert_to_grayscale(annotated_frame)

#         for indices in POLYGONS_INDICES:
#             polygon = target_xy[indices].astype(int)

#             annotated_frame = sv.draw_filled_polygon(
#                 scene=annotated_frame,
#                 polygon=polygon,
#                 color=CYAN_COLOR,
#                 opacity=0.6
#             )

#         annotated_frame = projected_edge_annotator.annotate(
#             scene=annotated_frame,
#             key_points=projected_key_points)
#         annotated_frame = projected_vertex_annotator.annotate(
#             scene=annotated_frame,
#             key_points=projected_key_points)
#         annotated_frame = detected_vertex_annotator.annotate(
#             scene=annotated_frame,
#             key_points=detected_key_points)
#         sink.write_frame(annotated_frame)

In [ ]:
# %%capture
# !ffmpeg -y -i {TARGET_VIDEO_PATH} -vcodec libx264 -crf 28 {TARGET_VIDEO_COMPRESSED_PATH}

## track end segment the players

In [ ]:
%cd $HOME/segment-anything-2-real-time

In [ ]:
from sam2.build_sam import build_sam2_camera_predictor

SAM2_CHECKPOINT = "checkpoints/sam2.1_hiera_large.pt"
SAM2_CONFIG = "configs/sam2.1/sam2.1_hiera_l.yaml"

predictor = build_sam2_camera_predictor(SAM2_CONFIG, SAM2_CHECKPOINT)

In [ ]:
import cv2
import torch
from tqdm import tqdm

PLAYER_ID = 2
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7
INTERVAL = 30

mask_annotator = sv.MaskAnnotator(
    color=COLOR,
    color_lookup=sv.ColorLookup.TRACK,
    opacity=0.7)
box_annotator = sv.BoxAnnotator(
    color=COLOR,
    color_lookup=sv.ColorLookup.TRACK,
    thickness=2
)

annotated_frames = []

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD)[0]
detections = sv.Detections.from_inference(result)
detections = detections[detections.class_id == PLAYER_ID]

XYXY = detections.xyxy
CLASS_ID = detections.class_id
TRACKE_ID = list(range(1, len(CLASS_ID) + 1))

detections = sv.Detections(
    xyxy=XYXY,
    tracker_id=np.array(TRACKE_ID)
)

annotated_frame = frame.copy()
annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
sv.plot_image(annotated_frame)

with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    predictor.load_first_frame(frame)

    for xyxy, tracker_id in zip(XYXY, TRACKE_ID):
        xyxy = np.array([xyxy])

        _, object_ids, mask_logits = predictor.add_new_prompt(
            frame_idx=0,
            obj_id=tracker_id,
            bbox=xyxy
        )

for index, frame in tqdm(enumerate(frame_generator)):

    with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
        tracker_ids, mask_logits = predictor.track(frame)
        tracker_ids = np.array(tracker_ids)
        masks = (mask_logits > 0.0).cpu().numpy()
        masks = np.squeeze(masks).astype(bool)

        detections = sv.Detections(
            xyxy=sv.mask_to_xyxy(masks=masks),
            mask=masks,
            tracker_id=tracker_ids
        )

    if index % INTERVAL == 0:

        annotated_frame = frame.copy()
        annotated_frame = mask_annotator.annotate(scene=annotated_frame, detections=detections)
        annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
        annotated_frames.append(annotated_frame)

In [ ]:
images = annotated_frames[:5]
titles = [f"frame {index * INTERVAL}" for index in range(0, len(images))]

sv.plot_images_grid(
    images=images,
    grid_size=(5, 1),
    size=(5, 15),
    titles=titles
)

In [ ]:
sv.plot_image(annotated_frames[-1])

## clean noisy masks

In [ ]:
import numpy as np
import cv2

def filter_segments_by_area(mask: np.ndarray, area_threshold: int) -> np.ndarray:
    """
    Keeps the main segment and removes other segments below area_threshold.

    Args:
        mask (np.ndarray): Boolean mask.
        area_threshold (int): Minimum area to keep a segment.

    Returns:
        np.ndarray: Boolean mask after filtering.
    """
    assert mask.dtype == bool, "Input mask must be boolean."
    mask_uint8 = mask.astype(np.uint8)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask_uint8, connectivity=8)
    if num_labels <= 1:
        return mask.copy()
    main_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    filtered_mask = np.zeros_like(mask, dtype=bool)
    for label in range(1, num_labels):
        area = stats[label, cv2.CC_STAT_AREA]
        if label == main_label or area >= area_threshold:
            filtered_mask[labels == label] = True
    return filtered_mask


def filter_segments_by_distance(mask: np.ndarray, distance_threshold: float) -> np.ndarray:
    """
    Keeps the main segment and removes segments farther than distance_threshold.

    Args:
        mask (np.ndarray): Boolean mask.
        distance_threshold (float): Maximum allowed distance from the main segment.

    Returns:
        np.ndarray: Boolean mask after filtering.
    """
    assert mask.dtype == bool, "Input mask must be boolean."
    mask_uint8 = mask.astype(np.uint8)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_uint8, connectivity=8)
    if num_labels <= 1:
        return mask.copy()
    main_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    main_centroid = centroids[main_label]
    filtered_mask = np.zeros_like(mask, dtype=bool)
    for label in range(1, num_labels):
        centroid = centroids[label]
        dist = np.linalg.norm(centroid - main_centroid)
        if label == main_label or dist <= distance_threshold:
            filtered_mask[labels == label] = True
    return filtered_mask

In [ ]:
PLAYER_ID = 2
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7
INTERVAL = 30

mask_annotator = sv.MaskAnnotator(
    color=COLOR,
    color_lookup=sv.ColorLookup.TRACK,
    opacity=0.7)
box_annotator = sv.BoxAnnotator(
    color=COLOR,
    color_lookup=sv.ColorLookup.TRACK,
    thickness=2
)

annotated_frames_before = []
annotated_frames_after = []

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD)[0]
detections = sv.Detections.from_inference(result)
detections = detections[detections.class_id == PLAYER_ID]

XYXY = detections.xyxy
CLASS_ID = detections.class_id
TRACKER_ID = list(range(1, len(CLASS_ID) + 1))

detections = sv.Detections(
    xyxy=XYXY,
    tracker_id=np.array(TRACKER_ID)
)

with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    predictor.load_first_frame(frame)

    for xyxy, tracker_id in zip(XYXY, TRACKER_ID):
        xyxy = np.array([xyxy])

        _, object_ids, mask_logits = predictor.add_new_prompt(
            frame_idx=0,
            obj_id=tracker_id,
            bbox=xyxy
        )

for index, frame in tqdm(enumerate(frame_generator)):

    with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
        tracker_ids, mask_logits = predictor.track(frame)
        tracker_ids = np.array(tracker_ids)
        masks = (mask_logits > 0.0).cpu().numpy()
        masks = np.squeeze(masks).astype(bool)

        if index % INTERVAL == 0:

            masks_after = masks = np.array([
                filter_segments_by_distance(mask, distance_threshold=300)
                for mask
                in masks
            ])

            detections_before = sv.Detections(
                xyxy=sv.mask_to_xyxy(masks=masks),
                mask=masks,
                tracker_id=tracker_ids
            )
            detections_after = sv.Detections(
                xyxy=sv.mask_to_xyxy(masks=masks_after),
                mask=masks_after,
                tracker_id=tracker_ids
            )

            annotated_frame_before = frame.copy()
            annotated_frame_before = mask_annotator.annotate(scene=annotated_frame_before, detections=detections_before)
            annotated_frame_before = box_annotator.annotate(scene=annotated_frame_before, detections=detections_before)
            annotated_frames_before.append(annotated_frame_before)

            annotated_frame_after = frame.copy()
            annotated_frame_after = mask_annotator.annotate(scene=annotated_frame_after, detections=detections_after)
            annotated_frame_after = box_annotator.annotate(scene=annotated_frame_after, detections=detections_after)
            annotated_frames_after.append(annotated_frame_after)

In [ ]:
images_before = annotated_frames_before[:5]
images_after = annotated_frames_after[:5]

titles_before = [f"frame {index * INTERVAL} before" for index in range(len(images_before))]
titles_after = [f"frame {index * INTERVAL} after" for index in range(len(images_after))]

images = []
titles = []

for before_img, after_img, before_title, after_title in zip(images_before, images_after, titles_before, titles_after):
    images.append(before_img)
    titles.append(before_title)
    images.append(after_img)
    titles.append(after_title)

sv.plot_images_grid(
    images=images,
    grid_size=(5, 2),
    size=(10, 15),
    titles=titles
)

In [ ]:
sv.plot_image(annotated_frames_after[-1])

In [ ]:
# TARGET_VIDEO_PATH = SOURCE_VIDEO_PATH.parent / f"{SOURCE_VIDEO_PATH.stem}-masks{SOURCE_VIDEO_PATH.suffix}"
# TARGET_VIDEO_COMPRESSED_PATH = TARGET_VIDEO_PATH.parent / f"{TARGET_VIDEO_PATH.stem}-compressed{TARGET_VIDEO_PATH.suffix}"

# PLAYER_ID = 2
# CONFIDENCE_THRESHOLD = 0.3
# IOU_THRESHOLD = 0.7
# INTERVAL = 30

# mask_annotator = sv.MaskAnnotator(
#     color=COLOR,
#     color_lookup=sv.ColorLookup.TRACK,
#     opacity=0.7)
# box_annotator = sv.BoxAnnotator(
#     color=COLOR,
#     color_lookup=sv.ColorLookup.TRACK,
#     thickness=2
# )

# video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# frame = next(frame_generator)

# result = PLAYER_DETECTION_MODEL.infer(frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD)[0]
# detections = sv.Detections.from_inference(result)
# detections = detections[detections.class_id == PLAYER_ID]

# XYXY = detections.xyxy
# CLASS_ID = detections.class_id
# TRACKER_ID = list(range(1, len(CLASS_ID) + 1))

# detections = sv.Detections(
#     xyxy=XYXY,
#     tracker_id=np.array(TRACKER_ID)
# )

# with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
#     predictor.load_first_frame(frame)

#     for xyxy, tracker_id in zip(XYXY, TRACKER_ID):
#         xyxy = np.array([xyxy])

#         _, object_ids, mask_logits = predictor.add_new_prompt(
#             frame_idx=0,
#             obj_id=tracker_id,
#             bbox=xyxy
#         )

# with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
#     for index, frame in tqdm(enumerate(frame_generator)):

#         with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
#             tracker_ids, mask_logits = predictor.track(frame)
#             tracker_ids = np.array(tracker_ids)
#             masks = (mask_logits > 0.0).cpu().numpy()
#             masks = np.squeeze(masks).astype(bool)

#             masks = np.array([
#                 filter_segments_by_distance(mask, distance_threshold=300)
#                 for mask
#                 in masks
#             ])

#             detections = sv.Detections(
#                 xyxy=sv.mask_to_xyxy(masks=masks),
#                 mask=masks,
#                 tracker_id=tracker_ids
#             )

#             annotated_frame = frame.copy()
#             annotated_frame = convert_to_grayscale(annotated_frame)
#             annotated_frame = mask_annotator.annotate(scene=annotated_frame, detections=detections)
#             annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
#             sink.write_frame(annotated_frame)

In [ ]:
# %%capture
# !ffmpeg -y -i {TARGET_VIDEO_PATH} -vcodec libx264 -crf 28 {TARGET_VIDEO_COMPRESSED_PATH}

## detect players skeletons

In [ ]:
def plot_skeletons(key_points: sv.KeyPoints, tracker_ids: list[int], frame: np.array):
    for tracker_id in tracker_ids:
        tracker_key_points = key_points[int(tracker_id) - 1]
        annotator = sv.EdgeAnnotator(
            color=COLOR.by_idx(tracker_id),
            thickness=3
        )
        frame = annotator.annotate(scene=frame, key_points=tracker_key_points)
    return frame

In [ ]:
import torch
from transformers import AutoProcessor, VitPoseForPoseEstimation

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

POSE_ESTIMATION_MODEL_ID = "usyd-community/vitpose-plus-large"

pose_estimation_processor = AutoProcessor.from_pretrained(POSE_ESTIMATION_MODEL_ID)
pose_estimation_model = VitPoseForPoseEstimation.from_pretrained(
    POSE_ESTIMATION_MODEL_ID, device_map=DEVICE)

In [ ]:
import cv2
import torch
from tqdm import tqdm

PLAYER_ID = 2
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7
INTERVAL = 30

annotated_frames = []

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD)[0]
detections = sv.Detections.from_inference(result)
detections = detections[detections.class_id == PLAYER_ID]

XYXY = detections.xyxy
CLASS_ID = detections.class_id
TRACKE_ID = list(range(1, len(CLASS_ID) + 1))

detections = sv.Detections(
    xyxy=XYXY,
    tracker_id=np.array(TRACKE_ID)
)

with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    predictor.load_first_frame(frame)

    for xyxy, tracker_id in zip(XYXY, TRACKE_ID):
        xyxy = np.array([xyxy])

        _, object_ids, mask_logits = predictor.add_new_prompt(
            frame_idx=0,
            obj_id=tracker_id,
            bbox=xyxy
        )

for index, frame in tqdm(enumerate(frame_generator)):

    with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
        tracker_ids, mask_logits = predictor.track(frame)
        tracker_ids = np.array(tracker_ids)
        masks = (mask_logits > 0.0).cpu().numpy()
        masks = np.squeeze(masks).astype(bool)

        masks = np.array([
            filter_segments_by_distance(mask, distance_threshold=300)
            for mask
            in masks
        ])

        detections = sv.Detections(
            xyxy=sv.mask_to_xyxy(masks=masks),
            mask=masks,
            tracker_id=tracker_ids
        )

    if index % INTERVAL == 0:

        boxes = sv.xyxy_to_xywh(detections.xyxy)
        inputs = pose_estimation_processor(frame, boxes=[boxes], return_tensors="pt").to(DEVICE)

        # This is MOE architecture, we should specify dataset indexes for each image in range 0..5
        inputs["dataset_index"] = torch.tensor([0], device=DEVICE)

        with torch.no_grad():
            outputs = pose_estimation_model(**inputs)

        results = pose_estimation_processor.post_process_pose_estimation(outputs, boxes=[boxes])
        key_points = sv.KeyPoints.from_transformers(results[0])

        annotated_frame = frame.copy()
        annotated_frame = plot_skeletons(key_points=key_points, tracker_ids=detections.tracker_id, frame=annotated_frame)
        annotated_frames.append(annotated_frame)

In [ ]:
images = annotated_frames[:5]
titles = [f"frame {index * INTERVAL}" for index in range(0, len(images))]

sv.plot_images_grid(
    images=images,
    grid_size=(5, 1),
    size=(5, 15),
    titles=titles
)

In [ ]:
sv.plot_image(annotated_frames[-1])

## detect players ankles

In [ ]:
def plot_ankles(key_points: sv.KeyPoints, tracker_ids: list[int], frame: np.array):
    key_points = key_points[:, [15, 16]]
    h, w = frame.shape[:2]
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=(w, h))
    for tracker_id in tracker_ids:
        tracker_key_points = key_points[int(tracker_id) - 1]
        annotator = sv.VertexLabelAnnotator(
            color=COLOR.by_idx(tracker_id),
            text_color=sv.Color.BLACK,
            text_scale=text_scale,
            border_radius=2
        )
        frame = annotator.annotate(scene=frame, key_points=tracker_key_points, labels=["L", "R"])
    return frame

In [ ]:
import cv2
import torch
from tqdm import tqdm

PLAYER_ID = 2
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7
INTERVAL = 30

annotated_frames = []

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD)[0]
detections = sv.Detections.from_inference(result)
detections = detections[detections.class_id == PLAYER_ID]

XYXY = detections.xyxy
CLASS_ID = detections.class_id
TRACKE_ID = list(range(1, len(CLASS_ID) + 1))

detections = sv.Detections(
    xyxy=XYXY,
    tracker_id=np.array(TRACKE_ID)
)

with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    predictor.load_first_frame(frame)

    for xyxy, tracker_id in zip(XYXY, TRACKE_ID):
        xyxy = np.array([xyxy])

        _, object_ids, mask_logits = predictor.add_new_prompt(
            frame_idx=0,
            obj_id=tracker_id,
            bbox=xyxy
        )

for index, frame in tqdm(enumerate(frame_generator)):

    with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
        tracker_ids, mask_logits = predictor.track(frame)
        tracker_ids = np.array(tracker_ids)
        masks = (mask_logits > 0.0).cpu().numpy()
        masks = np.squeeze(masks).astype(bool)

        masks = np.array([
            filter_segments_by_distance(mask, distance_threshold=300)
            for mask
            in masks
        ])

        detections = sv.Detections(
            xyxy=sv.mask_to_xyxy(masks=masks),
            mask=masks,
            tracker_id=tracker_ids
        )

    if index % INTERVAL == 0:

        boxes = sv.xyxy_to_xywh(detections.xyxy)
        inputs = pose_estimation_processor(frame, boxes=[boxes], return_tensors="pt").to(DEVICE)

        # This is MOE architecture, we should specify dataset indexes for each image in range 0..5
        inputs["dataset_index"] = torch.tensor([0], device=DEVICE)

        with torch.no_grad():
            outputs = pose_estimation_model(**inputs)

        results = pose_estimation_processor.post_process_pose_estimation(outputs, boxes=[boxes])
        key_points = sv.KeyPoints.from_transformers(results[0])

        annotated_frame = frame.copy()
        annotated_frame = plot_ankles(key_points=key_points, tracker_ids=detections.tracker_id, frame=annotated_frame)
        annotated_frames.append(annotated_frame)

In [ ]:
images = annotated_frames[:5]
titles = [f"frame {index * INTERVAL}" for index in range(0, len(images))]

sv.plot_images_grid(
    images=images,
    grid_size=(5, 1),
    size=(5, 15),
    titles=titles
)

In [ ]:
sv.plot_image(annotated_frames[-1])

In [ ]:
# TARGET_VIDEO_PATH = SOURCE_VIDEO_PATH.parent / f"{SOURCE_VIDEO_PATH.stem}-pose{SOURCE_VIDEO_PATH.suffix}"
# TARGET_VIDEO_COMPRESSED_PATH = TARGET_VIDEO_PATH.parent / f"{TARGET_VIDEO_PATH.stem}-compressed{TARGET_VIDEO_PATH.suffix}"

# PLAYER_ID = 2
# CONFIDENCE_THRESHOLD = 0.3
# IOU_THRESHOLD = 0.7
# INTERVAL = 30

# video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# frame = next(frame_generator)

# result = PLAYER_DETECTION_MODEL.infer(frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD)[0]
# detections = sv.Detections.from_inference(result)
# detections = detections[detections.class_id == PLAYER_ID]

# XYXY = detections.xyxy
# CLASS_ID = detections.class_id
# TRACKER_ID = list(range(1, len(CLASS_ID) + 1))

# detections = sv.Detections(
#     xyxy=XYXY,
#     tracker_id=np.array(TRACKER_ID)
# )

# with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
#     predictor.load_first_frame(frame)

#     for xyxy, tracker_id in zip(XYXY, TRACKER_ID):
#         xyxy = np.array([xyxy])

#         _, object_ids, mask_logits = predictor.add_new_prompt(
#             frame_idx=0,
#             obj_id=tracker_id,
#             bbox=xyxy
#         )

# with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
#     for index, frame in tqdm(enumerate(frame_generator)):

#         with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
#             tracker_ids, mask_logits = predictor.track(frame)
#             tracker_ids = np.array(tracker_ids)
#             masks = (mask_logits > 0.0).cpu().numpy()
#             masks = np.squeeze(masks).astype(bool)

#             masks = np.array([
#                 filter_segments_by_distance(mask, distance_threshold=300)
#                 for mask
#                 in masks
#             ])

#             detections = sv.Detections(
#                 xyxy=sv.mask_to_xyxy(masks=masks),
#                 mask=masks,
#                 tracker_id=tracker_ids
#             )

#         try:
#             boxes = sv.xyxy_to_xywh(detections.xyxy)
#             inputs = pose_estimation_processor(frame, boxes=[boxes], return_tensors="pt").to(DEVICE)

#             # This is MOE architecture, we should specify dataset indexes for each image in range 0..5
#             inputs["dataset_index"] = torch.tensor([0], device=DEVICE)

#             with torch.no_grad():
#                 outputs = pose_estimation_model(**inputs)

#             results = pose_estimation_processor.post_process_pose_estimation(outputs, boxes=[boxes])
#             key_points = sv.KeyPoints.from_transformers(results[0])

#             annotated_frame = frame.copy()
#             annotated_frame = convert_to_grayscale(annotated_frame)
#             annotated_frame = plot_skeletons(key_points=key_points, tracker_ids=detections.tracker_id, frame=annotated_frame)
#             annotated_frame = plot_ankles(key_points=key_points, tracker_ids=detections.tracker_id, frame=annotated_frame)
#         except:
#             print("no keypoints")
#             annotated_frame = frame.copy()

#         sink.write_frame(annotated_frame)

In [ ]:
# %%capture
# !ffmpeg -y -i {TARGET_VIDEO_PATH} -vcodec libx264 -crf 28 {TARGET_VIDEO_COMPRESSED_PATH}

## calculate time in zone

In [ ]:
from typing import Dict
import numpy as np

class FPSBasedTimer:
    """
    A timer that calculates the duration each object has been detected based on frames
    per second (FPS). Timer resets when a tracker ID disappears for more than
    `max_missing_frames` frames.

    Attributes:
        fps (int): The frame rate of the video stream.
        frame_id (int): Current frame number.
        tracker_id2frame_id (Dict[int, int]): Maps each tracker's ID to the frame number
            of its most recent (re)appearance.
        tracker_id2misses (Dict[int, int]): Counts consecutive missed frames for each tracker.
        max_missing_frames (int): Number of allowed missed frames before removing a tracker.
    """

    def __init__(self, fps: int = 30, max_missing_frames: int = 5) -> None:
        self.fps = fps
        self.frame_id = 0
        self.max_missing_frames = max_missing_frames
        self.tracker_id2frame_id: Dict[int, int] = {}
        self.tracker_id2misses: Dict[int, int] = {}

    def tick(self, detections) -> np.ndarray:
        """
        Processes the current frame, updating time durations for each tracker.
        Only removes tracker after `max_missing_frames` consecutive missed frames.

        Args:
            detections: The current frame's detections. Must have .tracker_id attribute.

        Returns:
            np.ndarray: Time durations (in seconds) for each detection.
        """
        self.frame_id += 1
        detected_ids = set(detections.tracker_id)
        times = []

        # Update missing counts and remove IDs missing for too long
        for tracker_id in list(self.tracker_id2frame_id):
            if tracker_id not in detected_ids:
                self.tracker_id2misses[tracker_id] = self.tracker_id2misses.get(tracker_id, 0) + 1
                if self.tracker_id2misses[tracker_id] > self.max_missing_frames:
                    del self.tracker_id2frame_id[tracker_id]
                    del self.tracker_id2misses[tracker_id]
            else:
                self.tracker_id2misses[tracker_id] = 0

        # Add or refresh seen IDs
        for tracker_id in detected_ids:
            if tracker_id not in self.tracker_id2frame_id:
                self.tracker_id2frame_id[tracker_id] = self.frame_id

        # Calculate durations
        for tracker_id in detections.tracker_id:
            start_frame = self.tracker_id2frame_id[tracker_id]
            duration = (self.frame_id - start_frame) / self.fps
            times.append(duration)

        return np.array(times)

In [ ]:
def run_player_pose_estimation(frame, detections):
    boxes = sv.xyxy_to_xywh(detections.xyxy)
    inputs = pose_estimation_processor(frame, boxes=[boxes], return_tensors="pt").to(DEVICE)

    # This is MOE architecture, we should specify dataset indexes for each image in range 0..5
    inputs["dataset_index"] = torch.tensor([0], device=DEVICE)

    with torch.no_grad():
        outputs = pose_estimation_model(**inputs)

    results = pose_estimation_processor.post_process_pose_estimation(outputs, boxes=[boxes])
    return sv.KeyPoints.from_transformers(results[0])

In [ ]:
from sports.common.view import ViewTransformer
from sports.configs.basketball import BasketballCourtConfiguration

KEYPOINT_CONFIDENCE_THRESHOLD = 0.5
DETECTION_CONFIDENCE = 0.3
LEFT_PAINT_INDICES = [2, 3, 11, 9]
RIGHT_PAINT_INDICES = [21, 23, 30, 29]
POLYGONS_INDICES = [LEFT_PAINT_INDICES, RIGHT_PAINT_INDICES]

CONFIG = BasketballCourtConfiguration()
EDGES = [(p1 + 1, p2 + 1) for p1, p2 in CONFIG.edges]

def run_court_detection(frame):
    result = COURT_DETECTION_MODEL.infer(frame, confidence=DETECTION_CONFIDENCE)[0]
    key_points = sv.KeyPoints.from_inference(result)
    filter = key_points.confidence[0] > KEYPOINT_CONFIDENCE_THRESHOLD
    detected_key_points = key_points[:, filter]

    source_reference_xy = np.array(CONFIG.vertices)[filter]
    target_reference_xy = detected_key_points.xy[0]

    transformer = ViewTransformer(
        source=source_reference_xy,
        target=target_reference_xy
    )

    source_xy = np.array(CONFIG.vertices)
    target_xy = transformer.transform_points(points=source_xy)

    projected_key_points = sv.KeyPoints(
        xy=target_xy[np.newaxis, ...],
        confidence=key_points.confidence
    )
    return detected_key_points, projected_key_points


def run_paint_detection(projected_key_points):
    for indices in POLYGONS_INDICES:
        if all(projected_key_points[:, indices].confidence[0] > KEYPOINT_CONFIDENCE_THRESHOLD):
            return projected_key_points[:, indices].xy[0].astype(int)
    return None

In [ ]:
import numpy as np
from matplotlib.path import Path as MplPath

def is_in_polygon(polygon: np.ndarray, points: np.ndarray) -> np.ndarray:
    path = MplPath(polygon)
    return path.contains_points(points)

In [ ]:
PLAYER_ID = 2
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7
INTERVAL = 30

TARGET_VIDEO_PATH = SOURCE_VIDEO_PATH.parent / f"{SOURCE_VIDEO_PATH.stem}-time{SOURCE_VIDEO_PATH.suffix}"
TARGET_VIDEO_COMPRESSED_PATH = TARGET_VIDEO_PATH.parent / f"{TARGET_VIDEO_PATH.stem}-compressed{TARGET_VIDEO_PATH.suffix}"

# define annotators
detected_vertex_annotator = sv.VertexAnnotator(color=MAGENTA_COLOR, radius=8)
projected_vertex_annotator = sv.VertexAnnotator(color=CYAN_COLOR, radius=8)

projected_edge_annotator = sv.EdgeAnnotator(
    color=CYAN_COLOR,
    thickness=2,
    edges=EDGES
)

mask_not_violated_annotator = sv.MaskAnnotator(
    color=sv.Color.from_hex("#33ff99"),
    opacity=0.5,
    color_lookup=sv.ColorLookup.INDEX
)
label_not_violated_annotator = sv.LabelAnnotator(
    color=sv.Color.from_hex("#33ff99"),
    text_color=sv.Color.BLACK,
    text_scale=1,
    color_lookup=sv.ColorLookup.INDEX,
    text_position = sv.Position.BOTTOM_CENTER
)
mask_violated_annotator = sv.MaskAnnotator(
    color=sv.Color.from_hex("#ff66b2"),
    opacity=0.5,
    color_lookup=sv.ColorLookup.INDEX
)
label_violated_annotator = sv.LabelAnnotator(
    color=sv.Color.from_hex("#ff66b2"),
    text_color=sv.Color.BLACK,
    text_scale=1,
    color_lookup=sv.ColorLookup.INDEX,
    text_position = sv.Position.BOTTOM_CENTER
)
halo_annotator = sv.HaloAnnotator(
    color=sv.Color.WHITE,
    color_lookup=sv.ColorLookup.INDEX,
    opacity=0.5,
)

# setup timer
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

TIMER = FPSBasedTimer(
    fps=video_info.fps,
    max_missing_frames=video_info.fps // 4
)

frame = next(frame_generator)

# detect players
result = PLAYER_DETECTION_MODEL.infer(frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD)[0]
detections = sv.Detections.from_inference(result)
detections = detections[detections.class_id == PLAYER_ID]

XYXY = detections.xyxy
CLASS_ID = detections.class_id
TRACKER_ID = list(range(1, len(CLASS_ID) + 1))

detections = sv.Detections(
    xyxy=XYXY,
    tracker_id=np.array(TRACKER_ID)
)

# start tracking
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    predictor.load_first_frame(frame)

    for xyxy, tracker_id in zip(XYXY, TRACKER_ID):
        xyxy = np.array([xyxy])

        _, object_ids, mask_logits = predictor.add_new_prompt(
            frame_idx=0,
            obj_id=tracker_id,
            bbox=xyxy
        )

# track players
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for index, frame in tqdm(enumerate(frame_generator)):

        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            tracker_ids, mask_logits = predictor.track(frame)
            tracker_ids = np.array(tracker_ids)
            masks = (mask_logits > 0.0).cpu().numpy()
            masks = np.squeeze(masks).astype(bool)

            masks = np.array([
                filter_segments_by_distance(mask, distance_threshold=150)
                for mask
                in masks
            ])

            detections = sv.Detections(
                xyxy=sv.mask_to_xyxy(masks=masks),
                mask=masks,
                tracker_id=tracker_ids
            )

        # detect court and paint
        detected_key_points, projected_key_points = run_court_detection(frame)
        paint_polygon = run_paint_detection(projected_key_points)

        try:
            # detect feet
            player_key_points = run_player_pose_estimation(frame, detections)
            player_key_points = player_key_points[:, [15, 16]]

            # detect players in paint
            if paint_polygon is not None:
                x_is_in_polygon = is_in_polygon(polygon=paint_polygon, points=player_key_points.xy[:, 0])
                y_is_in_polygon = is_in_polygon(polygon=paint_polygon, points=player_key_points.xy[:, 1])
                detections_in_polygon = detections[x_is_in_polygon & y_is_in_polygon]
            else:
                detections_in_polygon = sv.Detections.empty()
        except:
            print("no keypoints")
            detections_in_polygon = sv.Detections.empty()

        if len(detections_in_polygon) == 0:
            detections_in_polygon.tracker_id = np.array([], dtype=int)

        # calculate time in paint
        time_in_polygon = np.array(TIMER.tick(detections_in_polygon))
        is_violated = time_in_polygon > 3
        detections_violated = detections_in_polygon[is_violated]
        time_violated = [f"{t:.2f}" for t in time_in_polygon[is_violated]]
        detections_not_violated = detections_in_polygon[~is_violated]
        time_not_violated = [f"{t:.2f}" for t in time_in_polygon[~is_violated]]

        # draw results
        annotated_frame = frame.copy()
        annotated_frame = convert_to_grayscale(annotated_frame)
        blank_frame = annotated_frame.copy()

        if paint_polygon is not None:
            annotated_frame = sv.draw_filled_polygon(
                scene=annotated_frame,
                polygon=paint_polygon,
                color=CYAN_COLOR,
                opacity=0.6
            )

        annotated_frame = projected_edge_annotator.annotate(
            scene=annotated_frame,
            key_points=projected_key_points)
        annotated_frame = projected_vertex_annotator.annotate(
            scene=annotated_frame,
            key_points=projected_key_points)
        annotated_frame = detected_vertex_annotator.annotate(
            scene=annotated_frame,
            key_points=detected_key_points)
        annotated_frame = replace_masked_regions(
            source_frame=blank_frame,
            target_frame=annotated_frame,
            masks=masks
        )
        annotated_frame = mask_violated_annotator.annotate(
            scene=annotated_frame,
            detections=detections_violated
        )
        annotated_frame = mask_not_violated_annotator.annotate(
            scene=annotated_frame,
            detections=detections_not_violated
        )

        annotated_frame = label_violated_annotator.annotate(
            scene=annotated_frame,
            detections=detections_violated,
            labels=time_violated
        )
        annotated_frame = label_not_violated_annotator.annotate(
            scene=annotated_frame,
            detections=detections_not_violated,
            labels=time_not_violated
        )

        sink.write_frame(annotated_frame)

In [ ]:
%%capture
!ffmpeg -y -i {TARGET_VIDEO_PATH} -vcodec libx264 -crf 28 {TARGET_VIDEO_COMPRESSED_PATH}

<div align="center">
  <p>
    Looking for more tutorials or have questions?
    Check out our <a href="https://github.com/roboflow/notebooks">GitHub repo</a> for more notebooks,
    or visit our <a href="https://discord.gg/GbfgXGJ8Bk">discord</a>.
  </p>
  
  <p>
    <strong>If you found this helpful, please consider giving us a ⭐
    <a href="https://github.com/roboflow/notebooks">on GitHub</a>!</strong>
  </p>

</div>